EXPLORE DATA

In [138]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return  relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [139]:
#Creating merged Dataframe for Toronto districts (Downtown, east, north, central)
torontodata_denc_venues = getNearbyVenues(names=df_torontodata_denc['Neighborhood'],
                                   latitudes=df_torontodata_denc['Latitude'],
                                   longitudes=df_torontodata_denc['Longitude']
                                  )

In [140]:
torontodata_denc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [141]:
#finding how many venues for each neighborhood
torontodata_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [142]:
#Analyzing data for each neighborhood

torontodata_denc_onehot = pd.get_dummies(torontodata_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontodata_denc_onehot['Neighborhood'] = torontodata_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontodata_denc_onehot.columns[-1]] + list(torontodata_denc_onehot.columns[:-1])
torontodata_denc_onehot = torontodata_denc_onehot[fixed_columns]

torontodata_denc_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
#Grouping using mean of frequency
torontodata_denc_grouped = torontodata_denc_onehot.groupby('Neighborhood').mean().reset_index()
torontodata_denc_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.142857,0.071429,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0


In [146]:
#Finding out top 5 common venues by neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns based on number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontodata_denc_grouped['Neighborhood']

for ind in np.arange(torontodata_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontodata_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bar,Bakery
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Coffee Shop,Rental Car Location,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Yoga Studio,Sushi Restaurant,Italian Restaurant


CLUSTERING DATA

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print ('all loaded')


all loaded


In [148]:
from sklearn.cluster import KMeans
print ('KMeans loaded')

KMeans loaded


In [149]:
# set number of clusters
kclusters = 5

torontodata_denc_grouped_clustering = torontodata_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontodata_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 2, 0, 0, 2, 0, 0, 2])

In [150]:
# add labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torontodata_denc_merged = df_torontodata_denc

# merge toronto_grouped with toronto_data to get coordinates for each neighborhood
torontodata_denc_merged = torontodata_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# last columns
torontodata_denc_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Pub,Health Food Store,Wings Joint,Cuban Restaurant
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Fast Food Restaurant,Sushi Restaurant,Fish & Chips Shop,Brewery,Restaurant
3,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Latin American Restaurant,Ice Cream Shop
4,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Eastern European Restaurant,Donut Shop


In [153]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [154]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        torontodata_denc_merged['Latitude'], 
        torontodata_denc_merged['Longitude'], 
        torontodata_denc_merged['Neighborhood'], 
        torontodata_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

EXAMINE CLUSTERS

In [155]:
torontodata_denc_merged.loc[torontodata_denc_merged['Cluster Labels'] == 0, torontodata_denc_merged.columns[[1] + list(range(5, torontodata_denc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Studio District,Café,Coffee Shop,Bakery,Latin American Restaurant,Ice Cream Shop
7,Davisville,Dessert Shop,Italian Restaurant,Gym,Sandwich Place,Coffee Shop
11,"St. James Town, Cabbagetown",Restaurant,Italian Restaurant,Café,Coffee Shop,Bakery
14,"Garden District, Ryerson",Café,Theater,Coffee Shop,Sporting Goods Shop,Burrito Place
15,St. James Town,Café,Gastropub,Coffee Shop,Cocktail Bar,Farmers Market
16,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Park
17,Central Bay Street,Coffee Shop,Café,Yoga Studio,Sushi Restaurant,Italian Restaurant
18,"Richmond, Adelaide, King",Coffee Shop,Café,Opera House,Japanese Restaurant,Fast Food Restaurant
20,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Japanese Restaurant,Restaurant,Beer Bar
21,"Commerce Court, Victoria Hotel",Café,Gastropub,Japanese Restaurant,Coffee Shop,Deli / Bodega


In [156]:
torontodata_denc_merged.loc[torontodata_denc_merged['Cluster Labels'] == 1, torontodata_denc_merged.columns[[1] + list(range(5, torontodata_denc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Roselawn,Garden,Wings Joint,Cuban Restaurant,Eastern European Restaurant,Donut Shop


In [101]:
torontodata_denc_merged.loc[torontodata_denc_merged['Cluster Labels'] == 2, torontodata_denc_merged.columns[[1] + list(range(5, torontodata_denc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Women's Store,Cupcake Shop
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,"India Bazaar, The Beaches West",Fast Food Restaurant,Pub,Sushi Restaurant,Sandwich Place,Brewery,Restaurant,Italian Restaurant,Burrito Place,Intersection,Ice Cream Shop
3,Studio District,Café,Coffee Shop,Gastropub,American Restaurant,Brewery,Bakery,Latin American Restaurant,Pet Store,Bar,Seafood Restaurant
6,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Miscellaneous Shop
7,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Italian Restaurant,Sushi Restaurant,Café,Seafood Restaurant,Brewery
9,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Sports Bar,Bank,Restaurant,Fried Chicken Joint,Bagel Shop
11,"St. James Town, Cabbagetown",Coffee Shop,Café,Bakery,Italian Restaurant,Pub,Pizza Place,Park,Restaurant,Playground,Butcher
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Dance Studio,Café,Bubble Tea Shop,Pub
13,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Theater,Café,Breakfast Spot,Restaurant,Chocolate Shop,Bank


In [157]:
torontodata_denc_merged.loc[torontodata_denc_merged['Cluster Labels'] == 3, torontodata_denc_merged.columns[[1] + list(range(5, torontodata_denc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,"Moore Park, Summerhill East",Playground,Trail,Gym,Wings Joint,Cuban Restaurant
10,Rosedale,Park,Playground,Trail,Creperie,Eastern European Restaurant


In [159]:
torontodata_denc_merged.loc[torontodata_denc_merged['Cluster Labels'] == 4, torontodata_denc_merged.columns[[1] + list(range(5, torontodata_denc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Lawrence Park,Park,Bus Line,Swim School,Eastern European Restaurant,Donut Shop


Assigning names to Clusters
Cluster 0 - Cafe area
Cluster 1 - Garden area
Cluster 2 - Restaurant and shop area
Cluster 3 - Parks & trail area
Cluster 4 - Park area